Imports

In [ ]:
from time import time
import logging
import matplotlib.pyplot as plt
#
import os
import cv2
import numpy as np
import math

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import fetch_lfw_people
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from scipy.cluster.vq import kmeans, vq
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import precision_recall_curve, roc_curve
from sklearn.preprocessing import label_binarize

In [ ]:
try:
    import cPickle as pickle
except ImportError:  # Python 3.x
    import pickle

Face and eyes detection functions

In [ ]:
def face_detection(image):
    #image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    haar_classifier = cv2.CascadeClassifier('C:/Users/Thomas/Documents/face_detection/haarcascades/haarcascade_frontalface_default.xml')
    face = haar_classifier.detectMultiScale(image,minNeighbors=6,minSize=(75, 75)) #oi parametroi allazoun analoga to dataset
    if (len(face)==1): #if 1 face is found
        (x,y,w,h) = face[0]
        return True, image[y:y+w, x:x+h], face[0]
    elif (len(face)==0): #if no faces are detected
        
        return False, "zero"
    else: #if more than 1 faces are detected, the biggest is kept (the smaller ones usually belong in the background)
        biggest=0
        pos=0
        for i,(x,y,w,h) in enumerate(face):
            if biggest<(h*w):
                biggest=h*w
                pos=i
        return True, image[y:y+w, x:x+h], face[pos]

In [ ]:
def face_detection(image):
    #image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    haar_classifier = cv2.CascadeClassifier('C:/Users/Thomas/Documents/face_detection/haarcascades/haarcascade_frontalface_default.xml')
    face = haar_classifier.detectMultiScale(image)#,scaleFactor=1.2, minNeighbors=5)
    if (len(face)==1): #if 1 face is found
        (x,y,w,h) = face[0]
        '''
        for (x,y,w,h) in face: 
            cv2.rectangle(image, (x,y), (x+w,y+h), (255,255,255), 3)
        plt.imshow(image)
        plt.show()
        '''
        return True, image[y:y+w, x:x+h], face[0]
    elif (len(face)==0): #if no faces are detected
        return False, "zero"
    else: #if more than 1 faces are detected
        return False, "more"

In [ ]:
def eye_detection(image):
    #image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    haar_classifier = cv2.CascadeClassifier('C:/Users/Thomas/Documents/face_detection/haarcascades/haarcascade_eye.xml')
    eyes = haar_classifier.detectMultiScale(image, minNeighbors=4, minSize=(30, 30))
    if (len(eyes)==2): #if 2 eyes are detected
        return True, eyes
    else:
        haar_classifier = cv2.CascadeClassifier('C:/Users/Thomas/Documents/face_detection/haarcascades/haarcascade_eye_tree_eyeglasses.xml')
        glasses = haar_classifier.detectMultiScale(image)
        if (len(glasses)==2):
            return True, glasses
        else:
            return False, "more or less"

In [ ]:
eigen_pr_curve=[]
fisher_pr_curve=[]
sift_pr_curve=[]
surf_pr_curve=[]
orb_pr_curve=[]
brisk_pr_curve=[]

In [ ]:
datapath=('C:/Users/Thomas/Documents/Datasets/lfw(250x250)/')
min_faces=30
faces=[]
labels=[]

In [ ]:
target_names=[] #all the names are saved here
total_photos_seen=0 
n_classes=0
folders = os.listdir(datapath)
for folder in folders:
    label = os.path.basename(folder)
    training_images_path = datapath + '/' + folder
    num_of_faces = len(os.listdir(training_images_path))
    #if num_of_faces>=min_faces:   #people with low number of faces are skipped
    target_names.append(label)
    n_classes=n_classes+1
    faces_per_person=0
    for image in os.listdir(training_images_path):
        total_photos_seen=total_photos_seen+1
        image_path = training_images_path + '/' + image
        training_image = cv2.imread(image_path)
        #face=cv2.cvtColor(training_image,cv2.COLOR_BGR2GRAY)
        #resized=cv2.resize(face,(224,224),interpolation=cv2.INTER_AREA)
        result_im=face_detection(training_image) #this function returns TRUE/FALSE, the cut image and the coordinates of where the cut was made
        if result_im[0]: #if TRUE (face detected) then the cut face goes into the array
            cut_face=result_im[1]
            faces.append(cut_face)
            labels.append(n_classes)

        '''
        if faces_per_person==45:
                break
        ''' 

In [ ]:
new_faces=[]
new_labels=[]

In [ ]:
rotated_faces=0
#unwanted_images=[3,110,113,166,238,245,349,398,444,903,978,1194,1218] 
for i in range(len(faces)):
    #if i not in unwanted_images: in case there are pictures that cannot be processed
    training_image = cv2.imread(faces[i])
    result_im=eye_detection(training_image) #finding the place of the eyes
    
    if result_im[0]: #process of finding the angle of the two eyes, and alighnment
        rotated_faces=rotated_faces+1
        (x1,y1,w1,h1)=result_im[1][0]
        (x2,y2,w2,h2)=result_im[1][1]
        eye1_center_x=x1+(w1/2)
        eye1_center_y=y1+(h1/2)
        eye2_center_x=x2+(w2/2)
        eye2_center_y=y2+(h2/2)
        if (eye1_center_x>eye2_center_y):
            deltaY = eye1_center_y - eye2_center_y 
            deltaX = eye1_center_x - eye2_center_x
        else:
            deltaY = eye2_center_y - eye1_center_y 
            deltaX = eye2_center_x - eye1_center_x
        my_radians = math.atan2(deltaY , deltaX)
        my_degrees = math.degrees(my_radians)

        #rotation
        rows, cols = training_image.shape[:2]
        M = cv2.getRotationMatrix2D((cols/2, rows/2), my_degrees, 1)
        img_rotated = cv2.warpAffine(training_image, M, (cols,rows))
        '''
        font = cv2.FONT_HERSHEY_SIMPLEX
        plt.text(0,0,i)
        plt.imshow(img_rotated)
        plt.show()
        '''

        #scaling (all images will become 200x200)
        #if before the conversion the image is smaller than 200x200 INTER_LINEAR will be used, else INTER_AREA will be used
        image_size=rows*cols        
        if (image_size<=40000):
            scaled_image=cv2.resize(img_rotated, (200, 200), interpolation=cv2.INTER_LINEAR)
        else:
            scaled_image=cv2.resize(img_rotated, (200, 200), interpolation=cv2.INTER_AREA)

        #normalization
        norm_img = np.zeros((200, 200))
        norm_img = cv2.normalize(scaled_image, norm_img, 0, 255, cv2.NORM_MINMAX)

        final_image = cv2.cvtColor(norm_img, cv2.COLOR_BGR2GRAY)

        new_faces.append(final_image)
        new_labels.append(labels[i])
        
faces=new_faces
labels=new_labels

Eigenfaces

In [ ]:
flat_faces = []

for face in faces:
    flat_faces.append(face.reshape(-1))        
flat_faces = np.array(flat_faces)
labels=np.array(labels)     

Y = label_binarize(labels, classes=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28])

X_train, X_test, y_train, y_test = train_test_split(flat_faces,Y,test_size=0.30,stratify=Y,shuffle=True,random_state = 42)

pca = PCA(n_components=142, svd_solver='randomized',whiten=True)


clf=SVC(class_weight='balanced',C=28.104130383007977,gamma=7.281641758356972,kernel='poly',degree=1)
pipe = Pipeline([('pca', pca),
                ('clf', clf)])
classifier1 = OneVsRestClassifier(pipe)
classifier1.fit(X_train, y_train)
y_score_svc = classifier1.decision_function(X_test)
eigen_pr_curve.append(("svc",y_test,y_score_svc))


knn = KNeighborsClassifier(n_neighbors=3,p=2,leaf_size=50)
pipe = Pipeline([('pca', pca),
                ('knn', knn)])
classifier2 = OneVsRestClassifier(pipe)
classifier2.fit(X_train, y_train)
try:
    y_score_knn = classifier2.decision_function(X_test)
except AttributeError:  # Python 3.x
    y_score_knn = classifier2.predict_proba(X_test)
eigen_pr_curve.append(("knn",y_test,y_score_knn))


mlp = MLPClassifier(solver='adam',hidden_layer_sizes=(333,200,496,347,495),alpha=0.08193742846113258,
                    learning_rate='invscaling', activation='tanh',momentum=0.5725450935194276,verbose=False, early_stopping=True)
pipe = Pipeline([('pca', pca),
                ('mlp', mlp)])
classifier3 = OneVsRestClassifier(pipe)
classifier3.fit(X_train, y_train)
try:
    y_score_mlp = classifier3.decision_function(X_test)
except AttributeError:  # Python 3.x
    y_score_mlp = classifier3.predict_proba(X_test)
eigen_pr_curve.append(("mlp",y_test,y_score_mlp))


logreg = LogisticRegression(class_weight='balanced',C=33.60545334855496,solver='sag')
pipe = Pipeline([('pca', pca),
                ('logreg', logreg)])
classifier4 = OneVsRestClassifier(pipe)
classifier4.fit(X_train, y_train)
try:
    y_score_logreg = classifier4.decision_function(X_test)
except AttributeError:  # Python 3.x
    y_score_logreg = classifier4.predict_proba(X_test)
eigen_pr_curve.append(("logreg",y_test,y_score_logreg))


linear_svm = LinearSVC(C=95.48139914999754,loss='hinge')
pipe = Pipeline([('pca', pca),
                ('linear_svm', linear_svm)])
classifier5 = OneVsRestClassifier(pipe)
classifier5.fit(X_train, y_train)
try:
    y_score_linsvm = classifier5.decision_function(X_test)
except AttributeError:  # Python 3.x
    y_score_linsvm = classifier5.predict_proba(X_test)
eigen_pr_curve.append(("linsvm",y_test,y_score_linsvm))


tree = DecisionTreeClassifier(random_state=0,min_samples_leaf=1,max_depth=15,min_impurity_decrease=0.021318938975762092,criterion='entropy',max_features=10,
                              max_leaf_nodes=None)
pipe = Pipeline([('pca', pca),
                ('tree', tree)])
classifier6 = OneVsRestClassifier(pipe)
classifier6.fit(X_train, y_train)
try:
    y_score_tree = classifier6.decision_function(X_test)
except AttributeError:  # Python 3.x
    y_score_tree = classifier6.predict_proba(X_test)
eigen_pr_curve.append(("tree",y_test,y_score_tree))


forrest = RandomForestClassifier(random_state=0,min_samples_leaf=3,max_depth=10,n_estimators=387,class_weight='balanced',bootstrap=True)
pipe = Pipeline([('pca', pca),
                ('forrest', forrest)])
classifier7 = OneVsRestClassifier(pipe)
classifier7.fit(X_train, y_train)
try:
    y_score_forrest = classifier7.decision_function(X_test)
except AttributeError:  # Python 3.x
    y_score_forrest = classifier7.predict_proba(X_test)
eigen_pr_curve.append(("forrest",y_test,y_score_forrest))


adaboost = AdaBoostClassifier(n_estimators=264,random_state=0,learning_rate=0.2131171118825734)
pipe = Pipeline([('pca', pca),
                ('adaboost', adaboost)])
classifier8 = OneVsRestClassifier(pipe)
classifier8.fit(X_train, y_train)
try:
    y_score_ada = classifier8.decision_function(X_test)
except AttributeError:  # Python 3.x
    y_score_ada = classifier8.predict_proba(X_test)
eigen_pr_curve.append(("ada",y_test,y_score_ada))


gnb = GaussianNB()
pipe = Pipeline([('pca', pca),
                ('gnb', gnb)])
classifier9 = OneVsRestClassifier(pipe)
classifier9.fit(X_train, y_train)
try:
    y_score_gnb = classifier9.decision_function(X_test)
except AttributeError:  # Python 3.x
    y_score_gnb = classifier9.predict_proba(X_test)
eigen_pr_curve.append(("gnb",y_test,y_score_gnb))


gpc = GaussianProcessClassifier(kernel=1.0*RBF(length_scale=0.840398547098756),random_state=0)
pipe = Pipeline([('pca', pca),
                ('gpc', gpc)])
classifier10 = OneVsRestClassifier(pipe)
classifier10.fit(X_train, y_train)
try:
    y_score_gpc = classifier10.decision_function(X_test)
except AttributeError:  # Python 3.x
    y_score_gpc = classifier10.predict_proba(X_test)
eigen_pr_curve.append(("gpc",y_test,y_score_gpc))


ridge = RidgeClassifier()
pipe = Pipeline([('pca', pca),
                ('ridge', ridge)])
classifier11 = OneVsRestClassifier(pipe)
classifier11.fit(X_train, y_train)
try:
    y_score_ridge = classifier11.decision_function(X_test)
except AttributeError:  # Python 3.x
    y_score_ridge = classifier11.predict_proba(X_test)
eigen_pr_curve.append(("ridge",y_test,y_score_ridge))

In [ ]:
with open('eigen_pr_curve.p', 'wb') as fp:
    pickle.dump(eigen_pr_curve, fp, protocol=pickle.HIGHEST_PROTOCOL)#DONE

Fisherfaces

In [ ]:
flat_faces = []

for face in faces:
    flat_faces.append(face.reshape(-1))        
flat_faces = np.array(flat_faces)
labels=np.array(labels)     

Y = label_binarize(labels, classes=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28])

X_train, X_test, y_train, y_test = train_test_split(flat_faces,Y,test_size=0.30,stratify=Y,shuffle=True,random_state = 42)

pca = PCA(n_components=142, svd_solver='randomized',whiten=True)
lda = LDA()

svc=SVC(kernel='linear', class_weight='balanced',C=3.9241808801638385,gamma=96.58400875342846,degree=6)
pipe = Pipeline([('pca', pca),
                 ('lda',lda),
                ('svc', svc)])
classifier1 = OneVsRestClassifier(pipe)
classifier1.fit(X_train, y_train)
y_score_svc = classifier1.decision_function(X_test)
fisher_pr_curve.append(("svc",y_test,y_score_svc))


knn = KNeighborsClassifier(n_neighbors=13,p=1,leaf_size=32)
model = Pipeline([('pca', pca),
                 ('lda',lda),
                ('knn', knn)])
classifier2 = OneVsRestClassifier(pipe)
classifier2.fit(X_train, y_train)
try:
    y_score_knn = classifier2.decision_function(X_test)
except AttributeError:  # Python 3.x
    y_score_knn = classifier2.predict_proba(X_test)
fisher_pr_curve.append(("knn",y_test,y_score_knn))


mlp = MLPClassifier(solver='adam',hidden_layer_sizes=(417,469,290,194),alpha=0.011508598976828089,
                    learning_rate='constant', activation='tanh',momentum=0.5812137485951651,verbose=False, early_stopping=True)
pipe = Pipeline([('pca', pca),
                 ('lda',lda),
                ('mlp', mlp)])
classifier3 = OneVsRestClassifier(pipe)
classifier3.fit(X_train, y_train)
try:
    y_score_mlp = classifier3.decision_function(X_test)
except AttributeError:  # Python 3.x
    y_score_mlp = classifier3.predict_proba(X_test)
fisher_pr_curve.append(("mlp",y_test,y_score_mlp))


logreg = LogisticRegression(C=26.23190488581573,solver='lbfgs',class_weight='balanced')
pipe = Pipeline([('pca', pca),
                 ('lda',lda),
                ('logreg', logreg)])
classifier4 = OneVsRestClassifier(pipe)
classifier4.fit(X_train, y_train)
try:
    y_score_logreg = classifier4.decision_function(X_test)
except AttributeError:  # Python 3.x
    y_score_logreg = classifier4.predict_proba(X_test)
fisher_pr_curve.append(("logreg",y_test,y_score_logreg))


linear_svm = LinearSVC(C=0.851388416540332,loss='hinge',class_weight='balanced',fit_intercept=False)
pipe = Pipeline([('pca', pca),
                 ('lda', lda),
                ('linear_svm', linear_svm)])
classifier5 = OneVsRestClassifier(pipe)
classifier5.fit(X_train, y_train)
try:
    y_score_linsvm = classifier5.decision_function(X_test)
except AttributeError:  # Python 3.x
    y_score_linsvm = classifier5.predict_proba(X_test)
fisher_pr_curve.append(("linsvm",y_test,y_score_linsvm))


tree = DecisionTreeClassifier(random_state=0,criterion='gini',min_impurity_decrease=0.03861521502801396,max_depth=19,min_samples_leaf=2,max_leaf_nodes=None)
pipe = Pipeline([('pca', pca),
                 ('lda',lda),
                ('tree', tree)])
classifier6 = OneVsRestClassifier(pipe)
classifier6.fit(X_train, y_train)
try:
    y_score_tree = classifier6.decision_function(X_test)
except AttributeError:  # Python 3.x
    y_score_tree = classifier6.predict_proba(X_test)
fisher_pr_curve.append(("tree",y_test,y_score_tree))


forrest = RandomForestClassifier(random_state=0,max_depth=9,min_samples_leaf=3,n_estimators=241,class_weight='balanced',bootstrap=True)
pipe = Pipeline([('pca', pca),
                 ('lda', lda),
                ('forrest', forrest)])
classifier7 = OneVsRestClassifier(pipe)
classifier7.fit(X_train, y_train)
try:
    y_score_forrest = classifier7.decision_function(X_test)
except AttributeError:  # Python 3.x
    y_score_forrest = classifier7.predict_proba(X_test)
fisher_pr_curve.append(("forrest",y_test,y_score_forrest))


adaboost = AdaBoostClassifier(n_estimators=174,random_state=0,learning_rate=0.28183708257574086)
pipe = Pipeline([('pca', pca),
                 ('lda', lda),
                ('adaboost', adaboost)])
classifier8 = OneVsRestClassifier(pipe)
classifier8.fit(X_train, y_train)
try:
    y_score_ada = classifier8.decision_function(X_test)
except AttributeError:  # Python 3.x
    y_score_ada = classifier8.predict_proba(X_test)
fisher_pr_curve.append(("ada",y_test,y_score_ada))


gnb = GaussianNB()
pipe = Pipeline([('pca', pca),
                 ('lda', lda),
                ('gnb', gnb)])
classifier9 = OneVsRestClassifier(pipe)
classifier9.fit(X_train, y_train)
try:
    y_score_gnb = classifier9.decision_function(X_test)
except AttributeError:  # Python 3.x
    y_score_gnb = classifier9.predict_proba(X_test)
fisher_pr_curve.append(("gnb",y_test,y_score_gnb))


gpc = GaussianProcessClassifier(kernel=1.0*RBF(length_scale=1.0687440294594166),random_state=0)
pipe=Pipeline([('pca', pca),
                ('lda',lda),
                ('gpc', gpc)])
classifier10 = OneVsRestClassifier(pipe)
classifier10.fit(X_train, y_train)
try:
    y_score_gpc = classifier10.decision_function(X_test)
except AttributeError:  # Python 3.x
    y_score_gpc = classifier10.predict_proba(X_test)
fisher_pr_curve.append(("gpc",y_test,y_score_gpc))


ridge = RidgeClassifier(alpha=0.5)
pipe = Pipeline([('pca', pca),
                ('lda',lda),
                ('ridge', ridge)])
classifier11 = OneVsRestClassifier(pipe)
classifier11.fit(X_train, y_train)
try:
    y_score_ridge = classifier11.decision_function(X_test)
except AttributeError:  # Python 3.x
    y_score_ridge = classifier11.predict_proba(X_test)
fisher_pr_curve.append(("ridge",y_test,y_score_ridge))

In [ ]:
with open('fisher_pr_curve.p', 'wb') as fp:
    pickle.dump(fisher_pr_curve, fp, protocol=pickle.HIGHEST_PROTOCOL)

SIFT

In [ ]:
faces = np.array(faces)
labels=np.array(labels)

des_list = []

#orb=cv2.ORB_create()
#brisk = cv2.BRISK_create(30)
#surf=cv2.xfeatures2d.SURF_create()
sift = cv2.xfeatures2d.SIFT_create(nOctaveLayers=3, contrastThreshold=0.03, edgeThreshold=10, sigma=1.6)

list_to_delete=[] 

print("Finding descriptors for ",len(faces)," images")
t0 = time()
for i,face in enumerate(faces):
    image_path=datapath

    #kpts, des = orb.detectAndCompute(im, None)
    #kpts, des = brisk.detectAndCompute(im, None)
    #kpts, des = surf.detectAndCompute(im, None)
    kpts, des = sift.detectAndCompute(face, None)

    if des is not None:
        des_list.append((image_path, des))
    else:
        list_to_delete.append(i)
            
labels=np.array(labels)
des_list=np.array(des_list)

Y = label_binarize(labels, classes=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21])

X_train, X_test, y_train, y_test = train_test_split(des_list,Y,test_size=0.3,stratify=Y,shuffle=True,random_state = 42)

descriptors = X_train[0][1]
for image_path, descriptor in X_train[1:]:
    descriptors = np.vstack((descriptors, descriptor))

descriptors_float = descriptors.astype(float) 

k = 200
voc, variance = kmeans(descriptors_float, k, 1)

im_features = np.zeros((len(X_train), k), "float32")
for i in range(len(X_train)):
    words, distance = vq(X_train[i][1],voc)
    for w in words:
        im_features[i][w] += 1

nbr_occurences = np.sum( (im_features > 0) * 1, axis = 0)
idf = np.array(np.log((1.0*len(X_train)+1) / (1.0*nbr_occurences + 1)), 'float32')

stdSlr = StandardScaler().fit(im_features)
im_features = stdSlr.transform(im_features)

########################################
svc=SVC(class_weight='balanced',C=68.05601024732132,gamma=19.032746691658865,kernel='linear',degree=3)    
classifier1 = OneVsRestClassifier(svc)
classifier1.fit(im_features,y_train)

linsvc = LinearSVC(C=13.273988533706948,loss='hinge',class_weight=None,fit_intercept=True)    
classifier2 = OneVsRestClassifier(linsvc)
classifier2.fit(im_features,y_train)

forrest = RandomForestClassifier(n_estimators = 290, random_state=30,max_depth=9,min_samples_leaf=2,class_weight='balanced',bootstrap=True)    
classifier3 = OneVsRestClassifier(forrest)
classifier3.fit(im_features,y_train)

logreg = LogisticRegression(class_weight=None,C=81.37904643771212,solver='newton-cg')
classifier4 = OneVsRestClassifier(logreg)
classifier4.fit(im_features,y_train)

tree = DecisionTreeClassifier(random_state=0,criterion='entropy',min_impurity_decrease=0.009033508409721459,max_depth=None,min_samples_leaf=5,
                          max_leaf_nodes=7,max_features=None)    
classifier5 = OneVsRestClassifier(tree)
classifier5.fit(im_features,y_train)

knn = KNeighborsClassifier(n_neighbors=10,p=2,leaf_size=41)    
classifier6 = OneVsRestClassifier(knn)
classifier6.fit(im_features,y_train)

mlp = MLPClassifier(solver='adam',hidden_layer_sizes=(453,498,225,316),alpha=0.03009184217630222, verbose=0, early_stopping=True,activation='tanh',momentum=0.43299330454632257,learning_rate='constant')    
classifier7 = OneVsRestClassifier(mlp)
classifier7.fit(im_features,y_train)

adaboost = AdaBoostClassifier(random_state=0,n_estimators=248,learning_rate=0.10949478469869828)
classifier8 = OneVsRestClassifier(adaboost)
classifier8.fit(im_features,y_train)

gnb = GaussianNB()
classifier9 = OneVsRestClassifier(gnb)
classifier9.fit(im_features,y_train)

gpc = GaussianProcessClassifier(kernel=1.0*RBF(length_scale=2.331930318829218),random_state=0)    
classifier10 = OneVsRestClassifier(gpc)
classifier10.fit(im_features,y_train)

ridge = RidgeClassifier(alpha=0.5,solver='saga')
classifier11 = OneVsRestClassifier(ridge)
classifier11.fit(im_features,y_train)

###########################################
descriptors = X_test[0][1]
for image_path, descriptor in X_test[1:]:
    descriptors = np.vstack((descriptors, descriptor))

test_features = np.zeros((len(X_test), k), "float32")
for i in range(len(X_test)):
    words, distance = vq(X_test[i][1],voc)
    for w in words:
        test_features[i][w] += 1

nbr_occurences = np.sum( (test_features > 0) * 1, axis = 0)
idf = np.array(np.log((1.0*len(X_test)+1) / (1.0*nbr_occurences + 1)), 'float32')

test_features = stdSlr.transform(test_features)

y_score_svc = classifier1.decision_function(test_features)
sift_pr_curve.append(("svc",y_test,y_score_svc))

try:
    y_score_linsvc = classifier2.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_linsvc = classifier2.predict_proba(test_features)
sift_pr_curve.append(("linsvc",y_test,y_score_linsvc))

try:
    y_score_forrest = classifier3.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_forrest = classifier3.predict_proba(test_features)
sift_pr_curve.append(("forrest",y_test,y_score_forrest))

try:
    y_score_logreg = classifier4.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_logreg = classifier4.predict_proba(test_features)
sift_pr_curve.append(("logreg",y_test,y_score_logreg))

try:
    y_score_tree = classifier5.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_tree = classifier5.predict_proba(test_features)
sift_pr_curve.append(("tree",y_test,y_score_tree))

try:
    y_score_knn = classifier6.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_knn = classifier6.predict_proba(test_features)
sift_pr_curve.append(("knn",y_test,y_score_knn))

try:
    y_score_mlp = classifier7.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_mlp = classifier7.predict_proba(test_features)
sift_pr_curve.append(("mlp",y_test,y_score_mlp))

try:
    y_score_adaboost = classifier8.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_adaboost = classifier8.predict_proba(test_features)
sift_pr_curve.append(("adaboost",y_test,y_score_adaboost))

try:
    y_score_gnb = classifier9.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_gnb = classifier9.predict_proba(test_features)
sift_pr_curve.append(("gnb",y_test,y_score_gnb))

try:
    y_score_gpc = classifier10.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_gpc = classifier10.predict_proba(test_features)
sift_pr_curve.append(("gpc",y_test,y_score_gpc))

try:
    y_score_ridge = classifier11.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_ridge = classifier11.predict_proba(test_features)
sift_pr_curve.append(("ridge",y_test,y_score_ridge))

In [ ]:
with open('sift_pr_curve.p', 'wb') as fp:
    pickle.dump(sift_pr_curve, fp, protocol=pickle.HIGHEST_PROTOCOL)

SURF

In [ ]:
faces = np.array(faces)
labels=np.array(labels)

des_list = []

#orb=cv2.ORB_create()
#brisk = cv2.BRISK_create(30)
surf=cv2.xfeatures2d.SURF_create()
#sift = cv2.xfeatures2d.SIFT_create(nOctaveLayers=3, contrastThreshold=0.03, edgeThreshold=10, sigma=1.6)

list_to_delete=[] 

print("Finding descriptors for ",len(faces)," images")
t0 = time()
for i,face in enumerate(faces):
    image_path=datapath
    
    #kpts, des = orb.detectAndCompute(im, None)
    #kpts, des = brisk.detectAndCompute(im, None)
    kpts, des = surf.detectAndCompute(face, None)
    #kpts, des = sift.detectAndCompute(im, None)

    if des is not None:
        des_list.append((image_path, des))
    else:
        list_to_delete.append(i)
labels=np.array(labels)
des_list=np.array(des_list)

Y = label_binarize(labels, classes=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21])

X_train, X_test, y_train, y_test = train_test_split(des_list,Y,test_size=0.3,stratify=Y,shuffle=True,random_state = 42)

descriptors = X_train[0][1]
for image_path, descriptor in X_train[1:]:
    descriptors = np.vstack((descriptors, descriptor))

descriptors_float = descriptors.astype(float) 

k = 200
voc, variance = kmeans(descriptors_float, k, 1)

im_features = np.zeros((len(X_train), k), "float32")
for i in range(len(X_train)):
    words, distance = vq(X_train[i][1],voc)
    for w in words:
        im_features[i][w] += 1

nbr_occurences = np.sum( (im_features > 0) * 1, axis = 0)
idf = np.array(np.log((1.0*len(X_train)+1) / (1.0*nbr_occurences + 1)), 'float32')

stdSlr = StandardScaler().fit(im_features)
im_features = stdSlr.transform(im_features)

########################################
svc=SVC(class_weight='balanced',C=68.05601024732132,gamma=19.032746691658865,kernel='linear',degree=3)    
classifier1 = OneVsRestClassifier(svc)
classifier1.fit(im_features,y_train)

linsvc = LinearSVC(C=13.273988533706948,loss='hinge',class_weight=None,fit_intercept=True)    
classifier2 = OneVsRestClassifier(linsvc)
classifier2.fit(im_features,y_train)

forrest = RandomForestClassifier(n_estimators = 290, random_state=30,max_depth=9,min_samples_leaf=2,class_weight='balanced',bootstrap=True)    
classifier3 = OneVsRestClassifier(forrest)
classifier3.fit(im_features,y_train)

logreg = LogisticRegression(class_weight=None,C=81.37904643771212,solver='newton-cg')
classifier4 = OneVsRestClassifier(logreg)
classifier4.fit(im_features,y_train)

tree = DecisionTreeClassifier(random_state=0,criterion='entropy',min_impurity_decrease=0.009033508409721459,max_depth=None,min_samples_leaf=5,
                          max_leaf_nodes=7,max_features=None)    
classifier5 = OneVsRestClassifier(tree)
classifier5.fit(im_features,y_train)

knn = KNeighborsClassifier(n_neighbors=10,p=2,leaf_size=41)    
classifier6 = OneVsRestClassifier(knn)
classifier6.fit(im_features,y_train)

mlp = MLPClassifier(solver='adam',hidden_layer_sizes=(453,498,225,316),alpha=0.03009184217630222, verbose=0, early_stopping=True,activation='tanh',momentum=0.43299330454632257,learning_rate='constant')    
classifier7 = OneVsRestClassifier(mlp)
classifier7.fit(im_features,y_train)

adaboost = AdaBoostClassifier(random_state=0,n_estimators=248,learning_rate=0.10949478469869828)
classifier8 = OneVsRestClassifier(adaboost)
classifier8.fit(im_features,y_train)

gnb = GaussianNB()
classifier9 = OneVsRestClassifier(gnb)
classifier9.fit(im_features,y_train)

gpc = GaussianProcessClassifier(kernel=1.0*RBF(length_scale=2.331930318829218),random_state=0)    
classifier10 = OneVsRestClassifier(gpc)
classifier10.fit(im_features,y_train)

ridge = RidgeClassifier(alpha=0.5,solver='saga')
classifier11 = OneVsRestClassifier(ridge)
classifier11.fit(im_features,y_train)

###########################################
descriptors = X_test[0][1]
for image_path, descriptor in X_test[1:]:
    descriptors = np.vstack((descriptors, descriptor))

test_features = np.zeros((len(X_test), k), "float32")
for i in range(len(X_test)):
    words, distance = vq(X_test[i][1],voc)
    for w in words:
        test_features[i][w] += 1

nbr_occurences = np.sum( (test_features > 0) * 1, axis = 0)
idf = np.array(np.log((1.0*len(X_test)+1) / (1.0*nbr_occurences + 1)), 'float32')

test_features = stdSlr.transform(test_features)

y_score_svc = classifier1.decision_function(test_features)
surf_pr_curve.append(("svc",y_test,y_score_svc))

try:
    y_score_linsvc = classifier2.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_linsvc = classifier2.predict_proba(test_features)
surf_pr_curve.append(("linsvc",y_test,y_score_linsvc))

try:
    y_score_forrest = classifier3.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_forrest = classifier3.predict_proba(test_features)
surf_pr_curve.append(("forrest",y_test,y_score_forrest))

try:
    y_score_logreg = classifier4.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_logreg = classifier4.predict_proba(test_features)
surf_pr_curve.append(("logreg",y_test,y_score_logreg))

try:
    y_score_tree = classifier5.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_tree = classifier5.predict_proba(test_features)
surf_pr_curve.append(("tree",y_test,y_score_tree))

try:
    y_score_knn = classifier6.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_knn = classifier6.predict_proba(test_features)
surf_pr_curve.append(("knn",y_test,y_score_knn))

try:
    y_score_mlp = classifier7.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_mlp = classifier7.predict_proba(test_features)
surf_pr_curve.append(("mlp",y_test,y_score_mlp))

try:
    y_score_adaboost = classifier8.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_adaboost = classifier8.predict_proba(test_features)
surf_pr_curve.append(("adaboost",y_test,y_score_adaboost))

try:
    y_score_gnb = classifier9.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_gnb = classifier9.predict_proba(test_features)
surf_pr_curve.append(("gnb",y_test,y_score_gnb))

try:
    y_score_gpc = classifier10.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_gpc = classifier10.predict_proba(test_features)
surf_pr_curve.append(("gpc",y_test,y_score_gpc))

try:
    y_score_ridge = classifier11.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_ridge = classifier11.predict_proba(test_features)
surf_pr_curve.append(("ridge",y_test,y_score_ridge))

In [ ]:
with open('surf_pr_curve.p', 'wb') as fp:
    pickle.dump(surf_pr_curve, fp, protocol=pickle.HIGHEST_PROTOCOL)

BRISK

In [ ]:
faces = np.array(faces)
labels=np.array(labels)

des_list = []

#orb=cv2.ORB_create()
brisk = cv2.BRISK_create(30)
#surf=cv2.xfeatures2d.SURF_create()
#sift = cv2.xfeatures2d.SIFT_create(nOctaveLayers=3, contrastThreshold=0.03, edgeThreshold=10, sigma=1.6)

list_to_delete=[] 

print("Finding descriptors for ",len(faces)," images")
t0 = time()
for i,face in enumerate(faces):
    image_path=datapath

    #kpts, des = orb.detectAndCompute(im, None)
    kpts, des = brisk.detectAndCompute(face, None)
    #kpts, des = surf.detectAndCompute(im, None)
    #kpts, des = sift.detectAndCompute(im, None)

    if des is not None:
        des_list.append((image_path, des))
    else:
        list_to_delete.append(i)
            
new_faces = [j for i, j in enumerate(faces) if i not in list_to_delete]
faces=new_faces

new_labels = [j for i, j in enumerate(labels) if i not in list_to_delete]
labels=new_labels

labels=np.array(labels)
des_list=np.array(des_list)

Y = label_binarize(labels, classes=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21])

X_train, X_test, y_train, y_test = train_test_split(des_list,Y,test_size=0.3,stratify=Y,shuffle=True,random_state = 42)

descriptors = X_train[0][1]
for image_path, descriptor in X_train[1:]:
    descriptors = np.vstack((descriptors, descriptor))

descriptors_float = descriptors.astype(float) 

k = 200
voc, variance = kmeans(descriptors_float, k, 1)

im_features = np.zeros((len(X_train), k), "float32")
for i in range(len(X_train)):
    words, distance = vq(X_train[i][1],voc)
    for w in words:
        im_features[i][w] += 1

nbr_occurences = np.sum( (im_features > 0) * 1, axis = 0)
idf = np.array(np.log((1.0*len(X_train)+1) / (1.0*nbr_occurences + 1)), 'float32')

stdSlr = StandardScaler().fit(im_features)
im_features = stdSlr.transform(im_features)

########################################
print("*")
svc=SVC(class_weight='balanced',C=68.05601024732132,gamma=19.032746691658865,kernel='linear',degree=3)    
classifier1 = OneVsRestClassifier(svc)
classifier1.fit(im_features,y_train)
print("*")
linsvc = LinearSVC(C=13.273988533706948,loss='hinge',class_weight=None,fit_intercept=True)    
classifier2 = OneVsRestClassifier(linsvc)
classifier2.fit(im_features,y_train)
print("*")
forrest = RandomForestClassifier(n_estimators = 290, random_state=30,max_depth=9,min_samples_leaf=2,class_weight='balanced',bootstrap=True)    
classifier3 = OneVsRestClassifier(forrest)
classifier3.fit(im_features,y_train)
print("*")
logreg = LogisticRegression(class_weight=None,C=81.37904643771212,solver='newton-cg')
classifier4 = OneVsRestClassifier(logreg)
classifier4.fit(im_features,y_train)
print("*")
tree = DecisionTreeClassifier(random_state=0,criterion='entropy',min_impurity_decrease=0.009033508409721459,max_depth=None,min_samples_leaf=5,
                          max_leaf_nodes=7,max_features=None)    
classifier5 = OneVsRestClassifier(tree)
classifier5.fit(im_features,y_train)
print("*")
knn = KNeighborsClassifier(n_neighbors=10,p=2,leaf_size=41)    
classifier6 = OneVsRestClassifier(knn)
classifier6.fit(im_features,y_train)
print("*")
mlp = MLPClassifier(solver='adam',hidden_layer_sizes=(453,498,225,316),alpha=0.03009184217630222, verbose=0, early_stopping=True,activation='tanh',momentum=0.43299330454632257,learning_rate='constant')    
classifier7 = OneVsRestClassifier(mlp)
classifier7.fit(im_features,y_train)
print("*")
adaboost = AdaBoostClassifier(random_state=0,n_estimators=248,learning_rate=0.10949478469869828)
classifier8 = OneVsRestClassifier(adaboost)
classifier8.fit(im_features,y_train)
print("*")
gnb = GaussianNB()
classifier9 = OneVsRestClassifier(gnb)
classifier9.fit(im_features,y_train)
print("*")
gpc = GaussianProcessClassifier(kernel=1.0*RBF(length_scale=2.331930318829218),random_state=0)    
classifier10 = OneVsRestClassifier(gpc)
classifier10.fit(im_features,y_train)
print("*")
ridge = RidgeClassifier(alpha=0.5,solver='saga')
classifier11 = OneVsRestClassifier(ridge)
classifier11.fit(im_features,y_train)
print("*")
###########################################
descriptors = X_test[0][1]
for image_path, descriptor in X_test[1:]:
    descriptors = np.vstack((descriptors, descriptor))

test_features = np.zeros((len(X_test), k), "float32")
for i in range(len(X_test)):
    words, distance = vq(X_test[i][1],voc)
    for w in words:
        test_features[i][w] += 1

nbr_occurences = np.sum( (test_features > 0) * 1, axis = 0)
idf = np.array(np.log((1.0*len(X_test)+1) / (1.0*nbr_occurences + 1)), 'float32')

test_features = stdSlr.transform(test_features)

y_score_svc = classifier1.decision_function(test_features)
brisk_pr_curve.append(("svc",y_test,y_score_svc))

try:
    y_score_linsvc = classifier2.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_linsvc = classifier2.predict_proba(test_features)
brisk_pr_curve.append(("linsvc",y_test,y_score_linsvc))

try:
    y_score_forrest = classifier3.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_forrest = classifier3.predict_proba(test_features)
brisk_pr_curve.append(("forrest",y_test,y_score_forrest))

try:
    y_score_logreg = classifier4.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_logreg = classifier4.predict_proba(test_features)
brisk_pr_curve.append(("logreg",y_test,y_score_logreg))

try:
    y_score_tree = classifier5.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_tree = classifier5.predict_proba(test_features)
brisk_pr_curve.append(("tree",y_test,y_score_tree))

try:
    y_score_knn = classifier6.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_knn = classifier6.predict_proba(test_features)
brisk_pr_curve.append(("knn",y_test,y_score_knn))

try:
    y_score_mlp = classifier7.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_mlp = classifier7.predict_proba(test_features)
brisk_pr_curve.append(("mlp",y_test,y_score_mlp))

try:
    y_score_adaboost = classifier8.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_adaboost = classifier8.predict_proba(test_features)
brisk_pr_curve.append(("adaboost",y_test,y_score_adaboost))

try:
    y_score_gnb = classifier9.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_gnb = classifier9.predict_proba(test_features)
brisk_pr_curve.append(("gnb",y_test,y_score_gnb))

try:
    y_score_gpc = classifier10.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_gpc = classifier10.predict_proba(test_features)
brisk_pr_curve.append(("gpc",y_test,y_score_gpc))

try:
    y_score_ridge = classifier11.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_ridge = classifier11.predict_proba(test_features)
brisk_pr_curve.append(("ridge",y_test,y_score_ridge))

In [ ]:
with open('brisk_pr_curvec.p', 'wb') as fp:
    pickle.dump(brisk_pr_curve, fp, protocol=pickle.HIGHEST_PROTOCOL)

ORB

In [ ]:
faces = np.array(faces)
labels=np.array(labels)

des_list = []

orb=cv2.ORB_create()
#brisk = cv2.BRISK_create(30)
#surf=cv2.xfeatures2d.SURF_create()
#sift = cv2.xfeatures2d.SIFT_create(nOctaveLayers=3, contrastThreshold=0.03, edgeThreshold=10, sigma=1.6)

list_to_delete=[] 

print("Finding descriptors for ",len(faces)," images")
t0 = time()
for i,face in enumerate(faces):
    image_path=datapath
    
    kpts, des = orb.detectAndCompute(face, None)
    #kpts, des = brisk.detectAndCompute(im, None)
    #kpts, des = surf.detectAndCompute(im, None)
    #kpts, des = sift.detectAndCompute(im, None)

    if des is not None:
        des_list.append((image_path, des))
    else:
        list_to_delete.append(i)
        
labels=np.array(labels)
des_list=np.array(des_list)

Y = label_binarize(labels, classes=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21])

X_train, X_test, y_train, y_test = train_test_split(des_list,Y,test_size=0.3,stratify=Y,shuffle=True,random_state = 42)

descriptors = X_train[0][1]
for image_path, descriptor in X_train[1:]:
    descriptors = np.vstack((descriptors, descriptor))

descriptors_float = descriptors.astype(float) 

k = 200
voc, variance = kmeans(descriptors_float, k, 1)

im_features = np.zeros((len(X_train), k), "float32")
for i in range(len(X_train)):
    words, distance = vq(X_train[i][1],voc)
    for w in words:
        im_features[i][w] += 1

nbr_occurences = np.sum( (im_features > 0) * 1, axis = 0)
idf = np.array(np.log((1.0*len(X_train)+1) / (1.0*nbr_occurences + 1)), 'float32')

stdSlr = StandardScaler().fit(im_features)
im_features = stdSlr.transform(im_features)

########################################
print("*")
svc=SVC(class_weight='balanced',C=68.05601024732132,gamma=19.032746691658865,kernel='linear',degree=3)    
classifier1 = OneVsRestClassifier(svc)
classifier1.fit(im_features,y_train)
print("*")
linsvc = LinearSVC(C=13.273988533706948,loss='hinge',class_weight=None,fit_intercept=True)    
classifier2 = OneVsRestClassifier(linsvc)
classifier2.fit(im_features,y_train)
print("*")
forrest = RandomForestClassifier(n_estimators = 290, random_state=30,max_depth=9,min_samples_leaf=2,class_weight='balanced',bootstrap=True)    
classifier3 = OneVsRestClassifier(forrest)
classifier3.fit(im_features,y_train)
print("*")
logreg = LogisticRegression(class_weight=None,C=81.37904643771212,solver='newton-cg')
classifier4 = OneVsRestClassifier(logreg)
classifier4.fit(im_features,y_train)
print("*")
tree = DecisionTreeClassifier(random_state=0,criterion='entropy',min_impurity_decrease=0.009033508409721459,max_depth=None,min_samples_leaf=5,
                          max_leaf_nodes=7,max_features=None)    
classifier5 = OneVsRestClassifier(tree)
classifier5.fit(im_features,y_train)
print("*")
knn = KNeighborsClassifier(n_neighbors=10,p=2,leaf_size=41)    
classifier6 = OneVsRestClassifier(knn)
classifier6.fit(im_features,y_train)
print("*")
mlp = MLPClassifier(solver='adam',hidden_layer_sizes=(453,498,225,316),alpha=0.03009184217630222, verbose=0, early_stopping=True,activation='tanh',momentum=0.43299330454632257,learning_rate='constant')    
classifier7 = OneVsRestClassifier(mlp)
classifier7.fit(im_features,y_train)
print("*")
adaboost = AdaBoostClassifier(random_state=0,n_estimators=248,learning_rate=0.10949478469869828)
classifier8 = OneVsRestClassifier(adaboost)
classifier8.fit(im_features,y_train)
print("*")
gnb = GaussianNB()
classifier9 = OneVsRestClassifier(gnb)
classifier9.fit(im_features,y_train)
print("*")
gpc = GaussianProcessClassifier(kernel=1.0*RBF(length_scale=2.331930318829218),random_state=0)    
classifier10 = OneVsRestClassifier(gpc)
classifier10.fit(im_features,y_train)
print("*")
ridge = RidgeClassifier(alpha=0.5,solver='saga')
classifier11 = OneVsRestClassifier(ridge)
classifier11.fit(im_features,y_train)
print("*")
###########################################
descriptors = X_test[0][1]
for image_path, descriptor in X_test[1:]:
    descriptors = np.vstack((descriptors, descriptor))

test_features = np.zeros((len(X_test), k), "float32")
for i in range(len(X_test)):
    words, distance = vq(X_test[i][1],voc)
    for w in words:
        test_features[i][w] += 1

nbr_occurences = np.sum( (test_features > 0) * 1, axis = 0)
idf = np.array(np.log((1.0*len(X_test)+1) / (1.0*nbr_occurences + 1)), 'float32')

test_features = stdSlr.transform(test_features)

y_score_svc = classifier1.decision_function(test_features)
orb_pr_curve.append(("svc",y_test,y_score_svc))

try:
    y_score_linsvc = classifier2.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_linsvc = classifier2.predict_proba(test_features)
orb_pr_curve.append(("linsvc",y_test,y_score_linsvc))

try:
    y_score_forrest = classifier3.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_forrest = classifier3.predict_proba(test_features)
orb_pr_curve.append(("forrest",y_test,y_score_forrest))

try:
    y_score_logreg = classifier4.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_logreg = classifier4.predict_proba(test_features)
orb_pr_curve.append(("logreg",y_test,y_score_logreg))

try:
    y_score_tree = classifier5.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_tree = classifier5.predict_proba(test_features)
orb_pr_curve.append(("tree",y_test,y_score_tree))

try:
    y_score_knn = classifier6.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_knn = classifier6.predict_proba(test_features)
orb_pr_curve.append(("knn",y_test,y_score_knn))

try:
    y_score_mlp = classifier7.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_mlp = classifier7.predict_proba(test_features)
orb_pr_curve.append(("mlp",y_test,y_score_mlp))

try:
    y_score_adaboost = classifier8.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_adaboost = classifier8.predict_proba(test_features)
orb_pr_curve.append(("adaboost",y_test,y_score_adaboost))

try:
    y_score_gnb = classifier9.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_gnb = classifier9.predict_proba(test_features)
orb_pr_curve.append(("gnb",y_test,y_score_gnb))

try:
    y_score_gpc = classifier10.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_gpc = classifier10.predict_proba(test_features)
orb_pr_curve.append(("gpc",y_test,y_score_gpc))

try:
    y_score_ridge = classifier11.decision_function(test_features)
except AttributeError:  # Python 3.x
    y_score_ridge = classifier11.predict_proba(test_features)
orb_pr_curve.append(("ridge",y_test,y_score_ridge))

In [ ]:
with open('orb_pr_curve.p', 'wb') as fp:
    pickle.dump(orb_pr_curve, fp, protocol=pickle.HIGHEST_PROTOCOL)